In [1]:
import pandas as pd


KeyboardInterrupt



In [ ]:
from sklearn import preprocessing

scaler = preprocessing.MinMaxScaler()

In [ ]:
account_list = pd.read_csv('./1Kvertices-100Kedges/accounts.csv')

In [ ]:
account_list

In [ ]:
(account_list.CUSTOMER_ID == 'C_' + account_list.ACCOUNT_ID.astype(str)).all()

__Estrutura do `accounts.csv`__

- __ACCOUNT_ID__: identificador único.
- __CUSTOMER_ID__: outro identificador único
- __INIT_BALANCE__: saldo inicial da conta
- __COUNTRY__: país da conta
- __ACCOUNT_TYPE__: tipo de conta
- __IS_FRAUD__: Se é uma conta fraudulenta ou não
- __TX_BEHAVIOUR_ID__: ??

In [ ]:
transactions = pd.read_csv('./1Kvertices-100Kedges/transactions.csv')

In [ ]:
transactions

In [ ]:
transactions[transactions.TIMESTAMP == 1]

In [ ]:
def scaleColumns(df, cols_to_scale):
    for col in cols_to_scale:
        df[col] = pd.DataFrame(scaler.fit_transform(pd.DataFrame(df[col])),columns=[col])
    return df

In [ ]:
transactions[transactions.IS_FRAUD == True]

In [ ]:
transactions['SENDER_X'] = str(account_list[account_list.ACCOUNT_ID == 732].values[0])

In [ ]:
transactions

## TODO Melhorar tratamento de dados

In [ ]:
transactions['ALERT_ID'] = transactions['ALERT_ID'] + transactions['ALERT_ID']*100 + transactions['TX_AMOUNT']

In [ ]:
account_list['TX_BEHAVIOR_ID'] = account_list['TX_BEHAVIOR_ID']*100 + account_list['INIT_BALANCE']

## Fim TODO

In [ ]:
transactions[['TX_AMOUNT', 'ALERT_ID']] = scaler.fit_transform(transactions[['TX_AMOUNT', 'ALERT_ID']])
account_list[['INIT_BALANCE', 'TX_BEHAVIOR_ID']] = scaler.fit_transform(account_list[['INIT_BALANCE', 'TX_BEHAVIOR_ID']])

In [ ]:
account_list

In [ ]:
transactions

In [ ]:
sender_accs = transactions[transactions.TIMESTAMP == 0].SENDER_ACCOUNT_ID.values.tolist()

In [ ]:
sender_accs = sender_accs + transactions[transactions.TIMESTAMP == 0].RECEIVER_ACCOUNT_ID.values.tolist()

In [ ]:
accs = list(set(sender_accs))

In [ ]:
accs.sort()

In [ ]:
accs.index(accs[1])

In [ ]:
accs

In [ ]:
accs_t_0 = account_list.loc[accs].reset_index().reset_index()[['level_0', 'index' ,'INIT_BALANCE', 'TX_BEHAVIOR_ID']]

In [ ]:
accs_t_0

In [ ]:
def get_converted_index(old):
    return accs.index(old)

In [ ]:
transactions_t0 = transactions[transactions.TIMESTAMP == 0]

In [ ]:
transactions_t0.SENDER_ACCOUNT_ID = transactions_t0.SENDER_ACCOUNT_ID.apply(get_converted_index)

In [ ]:
transactions_t0

In [ ]:
transactions_t0.RECEIVER_ACCOUNT_ID = transactions_t0.RECEIVER_ACCOUNT_ID.apply(get_converted_index)

In [ ]:
transactions_t0

In [ ]:
edge_index = transactions_t0[['SENDER_ACCOUNT_ID', 'RECEIVER_ACCOUNT_ID']]

In [ ]:
edge_index.values

In [ ]:
edge_attr = transactions_t0[['TX_AMOUNT', 'TIMESTAMP', 'ALERT_ID']]

In [ ]:
edge_attr.values

In [ ]:
x = accs_t_0[['INIT_BALANCE', 'TX_BEHAVIOR_ID']].values

In [ ]:
x

In [ ]:
y = transactions_t0['IS_FRAUD']

In [ ]:
y = y.apply(lambda v: 1 if v else 0)

In [ ]:
y.values

In [ ]:
import torch
from torch_geometric.data import Data


def get_converted_index(old, acc_list):
    return acc_list.index(old)


graphs = []

for timestamp in range(200):
    timestamp_transacs = transactions[transactions.TIMESTAMP == timestamp]
    t_sender_accs = timestamp_transacs.SENDER_ACCOUNT_ID.values.tolist()
    t_receiver_accs = timestamp_transacs.RECEIVER_ACCOUNT_ID.values.tolist()
    timestamp_accs = list(set(t_sender_accs + t_receiver_accs))
    timestamp_accs.sort()
    t_accs_features = account_list.loc[timestamp_accs].reset_index()\
        .reset_index()[['level_0', 'index' ,'INIT_BALANCE', 'TX_BEHAVIOR_ID']]
    t_accs_features
    timestamp_transacs.SENDER_ACCOUNT_ID = timestamp_transacs.SENDER_ACCOUNT_ID\
        .apply(lambda line: get_converted_index(line, timestamp_accs))
    timestamp_transacs.RECEIVER_ACCOUNT_ID = timestamp_transacs.RECEIVER_ACCOUNT_ID\
        .apply(lambda line: get_converted_index(line, timestamp_accs))
    edge_index = torch.tensor(timestamp_transacs[['SENDER_ACCOUNT_ID', 'RECEIVER_ACCOUNT_ID']].values.T)
    edge_attr = torch.tensor(timestamp_transacs[['TX_AMOUNT', 'ALERT_ID']].values.T)
    y = torch.tensor(timestamp_transacs['IS_FRAUD'].apply(lambda v: 1 if v else 0).values)
    x = torch.tensor(t_accs_features[['INIT_BALANCE', 'TX_BEHAVIOR_ID']].values.T)
    edge_to_edge_adj_matr = torch.eye(edge_attr.T.shape[0])
    edge_to_node_adj_matr = torch.zeros([edge_attr.T.shape[0], x.T.shape[0]])                                                         
    node_to_edge_adj_matr = torch.zeros([x.T.shape[0], edge_attr.T.shape[0]])
    node_to_node_adj_matr = torch.eye(x.T.shape[0])
    for n, ei in enumerate(edge_index.T):
        node_to_node_adj_matr[ei[0]][ei[1]] = 1.0
        node_to_edge_adj_matr[ei[0]][n] = 1.0
        node_to_edge_adj_matr[ei[1]][n] = 1.0
        edge_to_node_adj_matr[n][ei[1]] = 1.0
        edge_to_node_adj_matr[n][ei[0]] = 1.0
    
    for n, ei in enumerate(edge_index.T):
        for n2, e2 in enumerate(edge_to_node_adj_matr[:, ei[0]]):
            if e2 == 1.0:
                edge_to_edge_adj_matr[n2][n] = 1.0
    data = Data(
        x=x,
        y=y,
        edge_attr=edge_attr,
        edge_index=edge_index,
        node_to_node_adj_matr=node_to_node_adj_matr,
        node_to_edge_adj_matr=node_to_edge_adj_matr,
        edge_to_node_adj_matr=edge_to_node_adj_matr,
        edge_to_edge_adj_matr=edge_to_edge_adj_matr
    )
    graphs.append(data)

In [ ]:
import os

NAME_DIR = '1Kvertices-100Kedges_pt/'

if not os.path.exists(NAME_DIR):
    os.mkdir(NAME_DIR)

NAME_DIR = '1Kvertices-100Kedges_pt/'

for i, g in enumerate(graphs):
    name_out = NAME_DIR + str(i) + '.pt'
    torch.save(g, name_out)